In [53]:
import cplex
from cplex.exceptions import CplexError
from docplex.mp.model import *
import random
import matplotlib.pyplot as plt
import docplex.mp.model as Solution
import docplex.cp.parameters as param
import numpy as np
import pandas as pd
import math as math
import csv

In [54]:
data = []
cost = []
t = [] # thời gian di chuyển từ điểm i đến điểm j

# Read the CSV file
with open('distance_matrix.csv', 'r', encoding="utf-8-sig") as file:
    csv_reader = csv.reader(file)

    # Iterate over each row in the CSV file
    for row in csv_reader:
        # Convert the elements in the row from strings to integers
        distance = [int(float(element)) for element in row]
        data.append(distance)
        distance = [int(float(element)) for element in row]
        t.append(distance)
        
    

Lấy các ràng buộc từ khách hàng

In [55]:
data_demands = []
ready_time = []
due_time = []
service_time = []

# Read the customer data from the text file
with open('data.txt', 'r') as file:
    # Skip the header line
    next(file)

    # Process each line in the file
    for line in file:
        # Remove leading/trailing whitespace and split the line into attributes
        attributes = line.strip().split(',')
        if len(attributes) >= 6:
        # Convert customer number, X coordinate, and Y coordinate to integers
            demand = int(attributes[3])
            redtime = int(attributes[4])
            dutime  =  int(attributes[5])
            sertime = int(attributes[6])

            # Add the dictionary to the data list
            data_demands.append(demand)
            ready_time.append(redtime)
            due_time.append(dutime)
            service_time.append(sertime)

Khai báo các chỉ số cơ bản

In [56]:
# n  = len(data) - 1 #số lượng khách hàng
n = 5 # số lượng khách hàng
Q = 30 #sức chứa của xe
K = 4 #số lượng xe
V = [i for i in range(1,K+1)] # set of vehicle
C = [i for i in range(1,n+1)] #set of client
N = [0] + C #set of node (warehouse + client)
q = {i:data_demands[i] for i in C} #yêu cầu của khách hàng i
s = {i:service_time[i] for i in C} #thời gian phục vụ khách hàng i
a = {i:ready_time[i] for i in C} #thời gian sớm nhất phục vụ khách hàng i
b = {i:due_time[i] for i in C} #thời gian muộn nhất phục vụ khách hàng i
sum(q.values())

70

In [57]:
A = [(i, j) for i in N for j in N if i != j]
c = {(i, j): data[i][j] for i, j in A}

In [58]:
mdl = Model('CVRP')

In [59]:
x = mdl.binary_var_dict(A, name='x')
u = mdl.continuous_var_dict(C, ub=Q, name='u') 


In [60]:
mdl.minimize(mdl.sum(c[i, j]*x[i,j] for i, j in A)) #1
mdl.add_constraints(mdl.sum(x[i, j] for j in N if j != i) == 1 for i in C ) #2
mdl.add_constraints(mdl.sum(x[i, j] for i in N if i != j) == 1 for j in C ) #3
mdl.add_indicator_constraints(
    mdl.indicator_constraint(
        x[i, j], u[i]+q[j] == u[j]
        ) 
        for i, j in A if i != 0 and j != 0 ) #4
mdl.add_constraints(u[i] >= q[i] for i in C) #5
mdl.parameters.timelimit(10)
solution = mdl.solve(log_output=True)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               10
Found incumbent of value 174.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 2 times.
MIP Presolve eliminated 7 rows and 11 columns.
MIP Presolve modified 6 coefficients.
Aggregator did 6 substitutions.
Reduced MIP has 14 rows, 30 columns, and 50 nonzeros.
Reduced MIP has 20 binaries, 0 generals, 0 SOSs, and 12 indicators.
Presolve time = 0.00 sec. (0.06 ticks)
Probing time = 0.00 sec. (0.02 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 14 rows, 30 columns, and 50 nonzeros.
Reduced MIP has 20 binaries, 0 generals, 0 SOSs, and 12 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.02 ticks)
Clique table members: 14.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 12 threads.
Root relaxation solutio

In [61]:
solution.display()

solution for: CVRP
objective: 109
status: OPTIMAL_SOLUTION(2)
x_0_2 = 1
x_0_3 = 1
x_0_5 = 1
x_1_0 = 1
x_2_0 = 1
x_3_4 = 1
x_4_1 = 1
x_5_0 = 1
u_1 = 30.000
u_2 = 30.000
u_3 = 10.000
u_4 = 20.000
u_5 = 10.000


In [62]:
active_arcs = [a for a in A if x[a].solution_value > 0.9]
active_arcs

[(0, 2), (0, 3), (0, 5), (1, 0), (2, 0), (3, 4), (4, 1), (5, 0)]

In [63]:
def find_all_routes(data_list):
    routes = []
    pre_value = -1
    current_route = []
    while True:
        if len(data_list) == 0:
            break
        for i in data_list:
            if pre_value == 0:
                routes.append(current_route.copy())
                current_route.clear()
                pre_value = -1
                break
            if i[0] == 0 and len(current_route) == 0:
                current_route.extend([i[0], i[1]])
                pre_value = i[1]
                data_list.remove(i)
                continue
            elif i[0] == pre_value:
                current_route.append(i[1])
                pre_value = i[1]
                data_list.remove(i)
                if len(data_list) == 0:
                    routes.append(current_route)
                    break
    print(routes)
    return routes


In [64]:
def format_route(route):
    return " -> ".join(str(node) for node in route)

In [65]:
result_routes = find_all_routes(active_arcs)
if result_routes == -1:
    print("No result")
else:
    print("solution for CVRP")
    print(f"objective: {solution.get_objective_value()}")
    for i, route in enumerate(result_routes, 1):
        print(f"Route {i}: {format_route(route)}")


[[0, 2, 0], [0, 3, 4, 1, 0], [0, 5, 0]]
solution for CVRP
objective: 109.0
Route 1: 0 -> 2 -> 0
Route 2: 0 -> 3 -> 4 -> 1 -> 0
Route 3: 0 -> 5 -> 0
